<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/Assignements/Part%205/Assignment_part_five.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DSML investigation:

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part five

### Due 29.10 (You get an extra hour!)

By now you should have 4 suspects left.
More information came in that suggests that the rogue agent is tampering with the sentiment annotation system of the SIMF which analyses news documents and marks their sentiment of intelligence analysis tasks.

This annotation is crutial to identify documents expressing negativity towards Switzerland and its allies.

Each document contains a column which shows which user accessed it. We know that the rogue agent accessed only the documents whose negative sentiment was high, and was then changed to positive or neutral. We will use a huggingface model to identify what records have been tampered with.


[You can find more models on this link](https://huggingface.co/models?sort=trending)


In [1]:
%%capture
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs
!pip install transformers[torch]
!pip install accelerate -U
# Import required packages

from transformers import pipeline, DataCollatorWithPadding
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

torch.cuda.is_available()


# 1. Getting to know our data

In [2]:
import pandas as pd
df = pd.read_excel('https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%205/data/Reduced_Set_2100.xlsx')

In [29]:
df.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   company     2100 non-null   object
 1   title       2100 non-null   object
 2   news        2100 non-null   object
 3   evaluation  2100 non-null   object
 4   year        2100 non-null   int64 
 5   month       2100 non-null   int64 
 6   day         2087 non-null   object
dtypes: int64(2), object(5)
memory usage: 115.0+ KB


# 2. Re-evaluating with SIMF's model:
Evaluate the sentiment on the title column using a sentiment pipeline trained on the `finiteautomata/bertweet-base-sentiment-analysis`model


_This may take a while_

In [4]:
from transformers import pipeline
# Your code here
sentiment_pipeline_simf = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")
titles = df.iloc[:, 1]
#intitialize
sentiments_simf = []
confidence_scores_simf = []

In [5]:
# Your code here
# Evaluate sentiment for each title
for title in titles:
    result = sentiment_pipeline_simf(title)
    
    # Assuming the model returns a single sentiment result
    sentiment = result[0]['label']
    confidence = result[0]['score']
    
    sentiments_simf.append(sentiment)
    confidence_scores_simf.append(confidence)


In [32]:
import pandas as pd


simf = pd.DataFrame({"Sentiment":sentiments_simf, "Score": confidence_scores_simf})
value_mapping = {
    'NEU': 'NEUTRAL',
    'NEG': 'NEGATIVE',
    'POS': 'POSITIVE'
}
simf['Sentiment'] = simf['Sentiment'].replace(value_mapping)

print(simf)

ValueError: If using all scalar values, you must pass an index

## 2.1 How many of the total entries match both the SIMF model **and** the hugginface model?

In [7]:
# Your code here
sentiment_pipeline_huggingface = pipeline("sentiment-analysis")
sentiment_huggingface = []
confidence_scores_huggingface = []
for title in titles:
    result = sentiment_pipeline_huggingface(title)
    
    # Assuming the model returns a single sentiment result
    sentiment = result[0]['label']
    confidence = result[0]['score']
    
    sentiment_huggingface.append(sentiment)
    confidence_scores_huggingface.append(confidence)

In [8]:
huggingface = pd.DataFrame({"Sentiment":sentiment_huggingface, "Score": confidence_scores_huggingface})
print(huggingface)

     Sentiment     Score
0     NEGATIVE  0.937911
1     POSITIVE  0.933045
2     NEGATIVE  0.999167
3     POSITIVE  0.995083
4     POSITIVE  0.994639
...        ...       ...
2095  NEGATIVE  0.998440
2096  NEGATIVE  0.992078
2097  POSITIVE  0.984533
2098  NEGATIVE  0.778633
2099  POSITIVE  0.999649

[2100 rows x 2 columns]


In [34]:
df['evaluation'] = df['evaluation'].str.upper()
matching_rows_count = len(df[df['evaluation'].isin(huggingface['Sentiment'])])
print(matching_rows_count)

1400


## 2.2 We will now focus on the entries that do not match
#### Identify all non matching entries

In [35]:
# Your code here
non_matching_entries = simf[~simf['Sentiment'].eq(huggingface['Sentiment'])]
# Print the non-matching entries
print("Non-matching entries:")
print(non_matching_entries)

Non-matching entries:
     Sentiment     Score
0      NEUTRAL  0.883557
1      NEUTRAL  0.966678
4      NEUTRAL  0.566068
5      NEUTRAL  0.554748
6      NEUTRAL  0.958037
...        ...       ...
2092   NEUTRAL  0.819136
2093   NEUTRAL  0.527811
2095   NEUTRAL  0.740490
2096   NEUTRAL  0.939423
2098  POSITIVE  0.542725

[1444 rows x 2 columns]


## 2.3 How many of those entries that our model predicted as negative, are evaluate as neutral or positive by the SIMF model ?

Store the resulting dataframe into a new one that we will be using in the following questions.

In [41]:
# Your code here
negative_huggingface = (huggingface.index[huggingface['Sentiment'] == 'NEGATIVE']).tolist()
neu_pos_simf = df[df['evaluation'].isin(['POSITIVE', 'NEUTRAL'])].index.tolist()
common_row_indices = list(set(negative_huggingface) & set(neu_pos_simf))

# Tampered data
tampered_df = df.iloc[common_row_indices]
print(tampered_df)

condition = ~df['evaluation'].isin(tampered_df['evaluation'])
non_tampered_df = df[condition]
print(non_tampered_df)


               company                                              title  \
2049           PEPSICO     BRONX ZOO FLIPPING OFF LIGHTS FOR HOLIDAY FEST   
2050   WAL MART STORES                             Fish dishes go further   
2051           PEPSICO           American giant tries to halve footprints   
2053  GENERAL ELECTRIC  Climate group seeks legal action over hoax web...   
2058  GENERAL ELECTRIC  Rogers defends climate stance as group challen...   
...                ...                                                ...   
2034   MERCK & COMPANY  TRENDS   RAIN-FOREST CHIC   Maybe Ben & Jerry'...   
2037   WAL MART STORES              Vancouver snub 'disappoints' Wal-Mart   
2041   WAL MART STORES  If you can't beat 'em...; Environmentalist Ada...   
2043   WAL MART STORES            Science needs to spark a bumper harvest   
2046  GENERAL ELECTRIC               Chief Turns to Webcast to Pitch G.E.   

                                                   news evaluation  year  \

# 3. Use the ChangeLog dataframe to identify the usersID's who edited the tampered entries, and only the altered entries

In [24]:
ChangeLog = pd.read_csv('https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%205/data/ChangeLog.csv')
ChangeLog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4169 entries, 0 to 4168
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UserID  4169 non-null   object
 1   title   4169 non-null   object
dtypes: object(2)
memory usage: 65.3+ KB


## 3.1 Identifying the users who have edited tampered documents

In [74]:
# Your code here
user_tampered = pd.merge(ChangeLog, tampered_df, on='title', how='inner')
print(user_tampered)



        UserID                                              title  \
0     [140991]  TRENDS   RAIN-FOREST CHIC   Maybe Ben & Jerry'...   
1     [628864]  TRENDS   RAIN-FOREST CHIC   Maybe Ben & Jerry'...   
2     [711683]  TRENDS   RAIN-FOREST CHIC   Maybe Ben & Jerry'...   
3     [398434]  TRENDS   RAIN-FOREST CHIC   Maybe Ben & Jerry'...   
4     [726073]  Food Corporate Giants Should Reverse the Bitte...   
...        ...                                                ...   
1322  [316450]  No twig left behind; Old into new. Wood waste ...   
1323  [255073]  No twig left behind; Old into new. Wood waste ...   
1324  [638950]  No twig left behind; Old into new. Wood waste ...   
1325  [829946]  GE takes Leap 56 to CF34 successor INTRODUCTIO...   
1326  [340001]  GE takes Leap 56 to CF34 successor INTRODUCTIO...   

               company                                               news  \
0      MERCK & COMPANY  […] Roddick told the Mexican conclave of big-b...   
1      MERCK & CO

## 3.2 Identifying the users who have edited non-tampered documents

In [77]:
# Your code here
user_non_tampered = ChangeLog.merge(non_tampered_df, on='title', how='inner')
print(user_non_tampered)



        UserID                                              title  \
0     [327047]  Tourists snap up British iPads to smuggle into...   
1     [401818]  Tourists snap up British iPads to smuggle into...   
2     [564061]  Tourists snap up British iPads to smuggle into...   
3     [446376]  Tourists snap up British iPads to smuggle into...   
4     [242912]           AFTER SEATTLE; Anarchists get organized.   
...        ...                                                ...   
1431  [484182]                                           IN BRIEF   
1432  [220420]                                        Corrections   
1433  [937535]                                        Corrections   
1434  [428637]                                        Corrections   
1435  [393146]                                        Corrections   

          company                                               news  \
0          APPLE   IT'S the digital version of the slow boat to C...   
1          APPLE   IT'S the

## 3.3 combining the results from `3.1` and `3.2` to identify users who only edited tampered documents.
These are our suspects.

In [176]:
# Your code here
rows_only_in_tampered= user_tampered[~user_tampered['UserID'].isin(user_non_tampered['UserID'])]

# Save the result to a new DataFrame
result_df = rows_only_in_tampered

# Print or do further processing with result_df
print(result_df)

target_user_id = '[527013]'

# Check if the target_user_id exists in the UserID column
if target_user_id in result_df['UserID'].values:
    print(f"UserID {target_user_id} found in the DataFrame.")
else:
    print(f"UserID {target_user_id} not found in the DataFrame.")

        UserID                                              title  \
8     [241404]  Letters: Anger simmers over energy bills; Thes...   
13    [868229]  Hammer isn't enough to buy their mine rights; ...   
17    [449172]  Run, don't walk, to greenhouse solutions; CAPI...   
49    [425350]  BHP seeks greener ways to mine; As carbon tax ...   
52    [231770]  Vast central Queensland LNG project may go int...   
...        ...                                                ...   
1248  [329120]     Rosneft and GE sign long-term service contract   
1258  [354752]  Pay the price to secure the planet before it's...   
1267  [806187]  Funds and pensions with $21 trillion pressure ...   
1272  [648842]   Paw Prints Disappearing for Siberia's Amur Tiger   
1278  [382851]  The greening of the computer; While the specia...   

               company                                               news  \
8      BANK OF AMERICA  […] Ending our addiction to fossil fuels, slas...   
13           CITI

# 4. Identifying important informations on the altered documents.

In this section we will use the TF-IDF text representation model to identify other important information on the altered documents.

In [146]:
# Make a list of the text within articles with the original dataset (the one of section 1)
tempered_news_list = user_tampered['news'].tolist()


In [147]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 1))

# Fit and transform the data to create the document-term matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(tempered_news_list)

# Get the feature names (vocabulary) used for the matrix
feature_names = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix to a DataFrame for visualization
document_term_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Visualize the DataFrame
print(document_term_df)


      000  000km  000mwh   02   04  042   07   08   09  09bn  ...  zespri  \
0     0.0    0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...     0.0   
1     0.0    0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...     0.0   
2     0.0    0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...     0.0   
3     0.0    0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...     0.0   
4     0.0    0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...     0.0   
...   ...    ...     ...  ...  ...  ...  ...  ...  ...   ...  ...     ...   
1322  0.0    0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...     0.0   
1323  0.0    0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...     0.0   
1324  0.0    0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...     0.0   
1325  0.0    0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...     0.0   
1326  0.0    0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...     0.0   

      zimbabwe  zip  zombo  zone  zones  zoning  zoo  zoologist  zurich  
0

In [153]:
# Keep the entries related to tampered documents
word_frequency = pd.DataFrame(document_term_df.sum()).reset_index()
# Identify the record that stands out the most on the altered documents (ou can use the sum of the tokenizers results)
word_frequency.columns = ['word', 'value']
word_frequency = word_frequency.sort_values('value',ascending=False)
print(word_frequency)



                   word      value
7791               said  43.256305
1595             carbon  40.460089
3249             energy  39.857131
3999                gas  36.038164
3195          emissions  30.574051
...                 ...        ...
344                 9pc   0.065978
7753  ruckversicherungs   0.065978
5971          munchener   0.065978
1651        catchphrase   0.065978
3521            express   0.065978

[9894 rows x 2 columns]
704


In [167]:
# How many records contain the word that stands out the most?
# e.g. if the word that stood out the most was "mouton", how many of the altered records contain the word mouton.
print(document_term_df[document_term_df[word_frequency['word'].to_list()[0]] != 0].shape[0])
# How about the second word that stands out the most
second_most_common_word = word_frequency['word'].iloc[1]
second_most_common_word_count = document_term_df[document_term_df[second_most_common_word] != 0].shape[0]
print(second_most_common_word_count)
# How about the third ?
third_most_common_word = word_frequency['word'].iloc[2]
third_most_common_word_count = document_term_df[document_term_df[third_most_common_word] != 0].shape[0]
print(third_most_common_word_count)
# How about the fourth ?
fourth_most_common_word = word_frequency['word'].iloc[3]
fourth_most_common_word_count = document_term_df[document_term_df[fourth_most_common_word] != 0].shape[0]
print(fourth_most_common_word_count)

704
424
560
449


#### Moodle quizz: if the order of frequency in appearance, did not match the values assigned by the tokenizer, is it normal?